## Vamos refatorar todo o código e usar POO:

## Classe: CreditCardValidator:

In [74]:
"""Módulo para validação e processamento de informações de cartão de crédito."""
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from utils.Config import Config
import re

class CreditCardValidator:
    """Validador de informações de cartão de crédito usando Azure Document Intelligence."""

    def __init__(self):
        """Inicializa o validador com as credenciais da Azure."""
        self.credential = AzureKeyCredential(Config.KEY)
        self.document_client = DocumentIntelligenceClient(Config.ENDPOINT, self.credential)

    def _validate_card_number(self, card_number: str) -> bool:
        """Valida o formato do número do cartão de crédito.

        Args:
            card_number: O número do cartão de crédito.

        Returns:
            True se o número do cartão for válido, False caso contrário.
        """
        clean_card_number = card_number.replace(" ", "")
        return len(clean_card_number) >= 10 and clean_card_number.isdigit()

    def _validate_expiration_date(self, expiration_date: str) -> bool:
        """Valida o formato da data de expiração do cartão de crédito (MM/YY).

        Args:
            expiration_date: A data de expiração no formato MM/YY.

        Returns:
            True se a data de expiração for válida, False caso contrário.
        """
        pattern = r"^(0[1-9]|1[0-2])/\d{2}$"
        return bool(re.match(pattern, expiration_date))

    def validate_card_info(self, card_info: dict) -> dict:
        """Valida as informações do cartão de crédito.

        Args:
            card_info: Um dicionário contendo as informações do cartão.

        Returns:
            Um dicionário com o resultado da validação ('is_valid': True/False).
        """
        card_number = card_info.get('CardNumber', '').replace(" ", "")
        expiration_date = card_info.get('ExpirationDate', '')

        # Verifique se cada campo de informação foi encontrado e validado
        if not card_number or not expiration_date:
            print("Número do cartão ou data de expiração não fornecidos.")
            return {"is_valid": False}

        is_valid = all([
            self._validate_card_number(card_number),
            self._validate_expiration_date(expiration_date)
        ])
        return {"is_valid": is_valid}

    def detect_credit_card_info_from_url(self, card_url: str) -> dict:
        """Detecta informações do cartão de crédito a partir de uma URL da imagem.

        Args:
            card_url: A URL da imagem do cartão de crédito no Azure Blob Storage.

        Returns:
            Um dicionário contendo as informações do cartão detectadas, ou None se ocorrer um erro.
        """
        try:
            card_info = self.document_client.begin_analyze_document(
                "prebuilt-creditCard", AnalyzeDocumentRequest(url_source=card_url)
            ).result()
            for document in card_info.documents:
                fields = document.fields
                return {
                    "card_name": fields.get("CardHolderName", {}).get("content", ""),
                    "card_number": fields.get("CardNumber", {}).get("content", "").replace(" ", ""),
                    "expiry_date": fields.get("ExpirationDate", {}).get("content", ""),
                    "bank_name": fields.get("IssuingBank", {}).get("content", "")
                }
            return None  # Nenhum documento encontrado
        except Exception as e:
            print(f"Erro na detecção de cartão: {e}")
            return None


* **A Classe basicamente possui dois métodos, um para obter os valores do cartão de crédito e o outro para valicação.**
* **Bora testar**:

In [75]:
#instanciar a classe: 
cd = CreditCardValidator()

In [76]:
# obter as informações do cartão:
cd.detect_credit_card_info_from_url("https://stdiolab2.blob.core.windows.net/cartoes/nubank.jpg")

{'card_name': 'GABRIEL LIMA',
 'card_number': '5032933437649846',
 'expiry_date': '09/17',
 'bank_name': 'ny\nbank'}

In [72]:
# Carregar as informações em uma variável:
card_info = cd.detect_credit_card_info_from_url("https://stdiolab2.blob.core.windows.net/cartoes/nubank.jpg")

In [77]:
# Fazer a validação do cartão:
cd.validate_card_info(card_info)

{'is_valid': True}

* **Perfeito!**
* Bora continuar"

# CreditCardValidator

A classe `CreditCardValidator` valida informações de cartões de crédito usando a API Azure Document Intelligence. Ela realiza a detecção e validação do número do cartão e da data de expiração, fornecendo uma resposta de validação que indica se as informações fornecidas são válidas.

## Sumário
- [Instalação](#instalação)
- [Uso da Classe](#uso-da-classe)
- [Métodos](#métodos)
  - [`__init__`](#__init__)
  - [`_validate_card_number`](#_validate_card_number)
  - [`_validate_expiration_date`](#_validate_expiration_date)
  - [`validate_card_info`](#validate_card_info)
  - [`detect_credit_card_info_from_url`](#detect_credit_card_info_from_url)


## Instalação

Para usar a `CreditCardValidator`, instale as bibliotecas necessárias, como `azure-core` e `azure-ai-documentintelligence`, usando o seguinte comando:

```bash
pip install azure-core azure-ai-documentintelligence
```

## Uso da Classe

A `CreditCardValidator` permite:
1. Validar se um número de cartão e uma data de expiração seguem os formatos esperados.
2. Detectar automaticamente informações de um cartão de crédito a partir de uma URL de imagem no Azure Blob Storage e validar essas informações.

## Métodos

### `__init__`

```python
def __init__(self)
```

Inicializa o `CreditCardValidator` com as credenciais da Azure, que são definidas na configuração (`Config.KEY` e `Config.ENDPOINT`). 

#### Exceções
- Pode lançar uma exceção se as credenciais da Azure estiverem incorretas.

---

### `_validate_card_number`

```python
def _validate_card_number(self, card_number: str) -> bool
```

Valida o número do cartão de crédito. Remove espaços do número do cartão e verifica se ele contém ao menos 10 dígitos numéricos.

#### Argumentos
- `card_number` (str): O número do cartão de crédito, que pode conter espaços.

#### Retorno
- `bool`: `True` se o número for válido; caso contrário, `False`.

---

### `_validate_expiration_date`

```python
def _validate_expiration_date(self, expiration_date: str) -> bool
```

Valida o formato da data de expiração do cartão de crédito no padrão `MM/YY`.

#### Argumentos
- `expiration_date` (str): A data de expiração no formato `MM/YY`.

#### Retorno
- `bool`: `True` se a data de expiração for válida; caso contrário, `False`.

---

### `validate_card_info`

```python
def validate_card_info(self, card_info: dict) -> dict
```

Valida as informações do cartão de crédito, como número do cartão e data de expiração. Retorna um dicionário indicando se as informações são válidas.

#### Argumentos
- `card_info` (dict): Um dicionário contendo as informações do cartão, com as chaves:
  - `CardNumber`: Número do cartão (str).
  - `ExpirationDate`: Data de expiração no formato `MM/YY` (str).

#### Retorno
- `dict`: Um dicionário com `{"is_valid": bool}` indicando se o cartão é válido (`True`) ou não (`False`).

### Observações
- **Dependências**: Certifique-se de que `Config.KEY` e `Config.ENDPOINT` estejam configurados corretamente para a autenticação da API da Azure.
- **Confiabilidade da Validação**: O código atual usa validações básicas de número de cartão e data de expiração. Para validações mais robustas, considere implementar o algoritmo de Luhn para o número do cartão.

Essa documentação oferece uma visão geral da `CreditCardValidator` e exemplos práticos para o uso correto da classe. Se houver dúvidas ou problemas adicionais, verifique as credenciais do Azure e os requisitos da API Document Intelligence.


## Classe: BlobStorageService:

In [66]:
"""Módulo para interação com o Azure Blob Storage."""
from utils.Config import Config
from azure.storage.blob import BlobServiceClient

class BlobStorageService:
    """Serviço para gerenciar o upload de arquivos para o Azure Blob Storage."""

    def __init__(self):
        """Inicializa o serviço com a string de conexão do Blob Storage."""
        self.blob_service_client = BlobServiceClient.from_connection_string(Config.STORAGE_CONNECTION)

    def upload_blob(self, file_path: str, file_name: str) -> str:
        """Envia um arquivo para o Azure Blob Storage.

        Args:
            file_path: O caminho local do arquivo.
            file_name: O nome do arquivo a ser salvo no Blob Storage.

        Returns:
            A URL do arquivo no Blob Storage, ou None se ocorrer um erro.
        """
        try:
            blob_client = self.blob_service_client.get_blob_client(container=Config.CONTAINER_NAME, blob=file_name)
            with open(file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)
            return blob_client.url
        except Exception as e:
            print(f"Erro no upload para Blob Storage: {e}")
            return None

In [67]:
# inicializar a classe:
blob = BlobStorageService()

In [68]:
blob.upload_blob("../img/nubank.jpg", 'test.jpg')

'https://stdiolab2.blob.core.windows.net/cartoes/test.jpg'

# BlobStorageService

A classe `BlobStorageService` facilita o upload de arquivos para o Azure Blob Storage. Utiliza a biblioteca `azure.storage.blob` para gerenciar a conexão e o envio dos arquivos.

## Sumário
- [Instalação](#instalação)
- [Uso da Classe](#uso-da-classe)
- [Métodos](#métodos)
  - [`__init__`](#__init__)
  - [`upload_blob`](#upload_blob)
- [Exemplo Completo de Uso](#exemplo-completo-de-uso)

## Instalação

Para usar a `BlobStorageService`, instale as dependências necessárias, especialmente a biblioteca `azure-storage-blob`:

```bash
pip install azure-storage-blob
```

## Uso da Classe

A `BlobStorageService` permite o upload de arquivos locais para um contêiner no Azure Blob Storage. Após o upload, a URL do arquivo no Blob Storage é retornada.

## Métodos

### `__init__`

```python
def __init__(self)
```

Inicializa a instância do `BlobStorageService` com a string de conexão do Blob Storage, que é especificada em `Config.STORAGE_CONNECTION`.

#### Exceções
- Pode lançar uma exceção se a `Config.STORAGE_CONNECTION` não for válida ou se a conexão ao Blob Storage falhar.

---

### `upload_blob`

```python
def upload_blob(self, file_path: str, file_name: str) -> str
```

Envia um arquivo local para um contêiner do Azure Blob Storage e retorna a URL do arquivo.

#### Argumentos
- `file_path` (str): O caminho local do arquivo a ser enviado.
- `file_name` (str): O nome do arquivo como será salvo no contêiner do Blob Storage.

#### Retorno
- `str`: A URL do arquivo no Blob Storage se o upload for bem-sucedido; caso contrário, `None`.

#### Exceções
- Em caso de erro no upload, o método imprime a mensagem de erro e retorna `None`.

---

## Exemplo Completo de Uso

Abaixo está um exemplo completo de uso da `BlobStorageService` para fazer o upload de um arquivo para o Azure Blob Storage.

```python
from utils.Config import Config  # Certifique-se de que Config está configurado corretamente
from azure.storage.blob import BlobServiceClient

# Inicialize o serviço de Blob Storage
blob_service = BlobStorageService()

# Caminho do arquivo local e nome do arquivo no Blob Storage
file_path = "path/to/your/file.txt"
file_name = "uploaded_file.txt"

# Upload do arquivo e obtenção da URL
file_url = blob_service.upload_blob(file_path, file_name)

if file_url:
    print("Arquivo enviado com sucesso:", file_url)
else:
    print("Falha no upload do arquivo.")
```

### Observações
- **Configuração**: As configurações de conexão (`Config.STORAGE_CONNECTION`) e de contêiner (`Config.CONTAINER_NAME`) devem estar corretamente configuradas para que o upload funcione.
- **Tratamento de Erros**: Em caso de falha, uma mensagem de erro é exibida e o método `upload_blob` retorna `None`.
- **Sobrescrita de Arquivos**: A função `upload_blob` usa `overwrite=True`, então arquivos com o mesmo nome serão sobrescritos no Blob Storage.


In [58]:
# configurando a path do banco de dados apenas para teste pois a variável já está setada na Config.py.
# Config.DATABASE_PATH = "card1.db"

## Classe: DatabaseService:  

In [59]:
"""Módulo para interação com o banco de dados SQLite."""
import sqlite3
from typing import Dict, List, Optional

from utils.Config import Config

class DatabaseService:
    """Serviço para gerenciar o banco de dados SQLite."""

    def __init__(self):
        """Inicializa o serviço com o caminho do banco de dados."""
        self.db_path = Config.DATABASE_PATH
        self._create_table()

    def _create_table(self):
        """Cria a tabela credit_cards se ela não existir."""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(
                """
                CREATE TABLE IF NOT EXISTS credit_cards (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    card_name TEXT,
                    card_number TEXT,
                    expiry_date TEXT,
                    bank_name TEXT,
                    is_valid TEXT,
                    processed_at TEXT
                )
                """
            )
            conn.commit()

    def _execute_query(self, query: str, params: tuple = None) -> Optional[sqlite3.Cursor]:
        """Executa uma query SQL com tratamento de erros.

        Args:
            query: A query SQL a ser executada.
            params: Os parâmetros da query (opcional).

        Returns:
            O cursor da query ou None caso ocorra um erro.
        """
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            try:
                if params:
                    cursor.execute(query, params)
                else:
                    cursor.execute(query)
                conn.commit()
                return cursor
            except sqlite3.Error as e:
                print(f"Erro na execução do SQL: {e}")
                return None

    def insert_card(self, card_info: Dict[str, str]) -> int:
        """Insere informações de um cartão de crédito no banco de dados.

        Args:
            card_info: Um dicionário com as informações do cartão.

        Returns:
            O ID do cartão inserido, ou None se ocorrer um erro.
        """
        query = """
        INSERT INTO credit_cards (card_name, card_number, expiry_date, bank_name, is_valid, processed_at)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        cursor = self._execute_query(query, (
            card_info["card_name"],
            card_info["card_number"],
            card_info["expiry_date"],
            card_info["bank_name"],
            card_info["is_valid"],
            card_info["processed_at"],
        ))
        if cursor:
          return cursor.lastrowid
        return None

    def get_all_cards(self) -> List[Dict[str, str]]:
        """Retorna todas as informações dos cartões do banco de dados.

        Returns:
            Uma lista de dicionários, onde cada dicionário representa um cartão.
        """
        query = "SELECT * FROM credit_cards"
        cursor = self._execute_query(query)
        if cursor:
          columns = [desc[0] for desc in cursor.description]
          return [dict(zip(columns, row)) for row in cursor.fetchall()]
        return []

    def get_card_by_id(self, card_id: int) -> Optional[Dict[str, str]]:
        """Retorna um cartão específico."""
        query = "SELECT * FROM credit_cards WHERE id = ?"
        cursor = self._execute_query(query, (card_id,))
        row = cursor.fetchone()
        if row:
            columns = [desc[0] for desc in cursor.description]
            return dict(zip(columns, row))
        return None
    
    def get_card_by_number(self, card_number: int) -> Optional[Dict[str, str]]:
        """Retorna um cartão específico."""
        query = "SELECT * FROM credit_cards WHERE card_number = ?"
        cursor = self._execute_query(query, (card_number,))
        row = cursor.fetchone()
        if row:
            columns = [desc[0] for desc in cursor.description]
            return dict(zip(columns, row))
        return None
    
    def execute_custom_query(self, query: str) -> List[Dict[str, str]]:
        """Executa uma consulta SQL personalizada."""
        if query.lower().startswith(('select')):
            cursor = self._execute_query(query)
            columns = [desc[0] for desc in cursor.description]
            return [dict(zip(columns, row)) for row in cursor.fetchall()]
        else:
            raise ValueError("Apenas consultas SELECT são permitidas")

    def update_card(self, card_id: int, card_info: Dict[str, str]) -> bool:
        """Atualiza um cartão existente."""
        query = """
        UPDATE credit_cards
        SET card_name = ?, card_number = ?, expiry_date = ?,
            bank_name = ?, is_valid = ?, processed_at = ?
        WHERE id = ?
        """
        cursor = self._execute_query(
            query,
            (
                card_info["card_name"],
                card_info["card_number"],
                card_info["expiry_date"],
                card_info["bank_name"],
                card_info["is_valid"],
                card_info["processed_at"],
                card_id,
            ),
        )
        return cursor.rowcount > 0

    def delete_card(self, card_id: int) -> bool:
        """Deleta um cartão."""
        query = "DELETE FROM credit_cards WHERE id = ?"
        cursor = self._execute_query(query, (card_id,))
        return cursor.rowcount > 0


In [60]:
# instanciar a classe :
db = DatabaseService()

In [62]:
# Realizando um teste e verificando se existem dados no bd.
db.get_all_cards()

[]

* **Como o banco foi criado agora não possui dados, mas a conexão e criação da tabela funcionou!**.

# DatabaseService

A classe `DatabaseService` permite interações com um banco de dados SQLite, incluindo operações CRUD (Create, Read, Update, Delete) para gerenciar informações de cartões de crédito.

## Sumário
- [Instalação](#instalação)
- [Uso da Classe](#uso-da-classe)
- [Métodos](#métodos)
  - [`__init__`](#__init__)
  - [`_create_table`](#_create_table)
  - [`_execute_query`](#_execute_query)
  - [`insert_card`](#insert_card)
  - [`get_all_cards`](#get_all_cards)
  - [`get_card_by_id`](#get_card_by_id)
  - [`get_card_by_number`](#get_card_by_number)
  - [`execute_custom_query`](#execute_custom_query)
  - [`update_card`](#update_card)
  - [`delete_card`](#delete_card)
- [Exemplo Completo de Uso](#exemplo-completo-de-uso)

## Instalação

A classe `DatabaseService` requer a biblioteca padrão do Python `sqlite3` para o banco de dados SQLite. Essa biblioteca é inclusa automaticamente em versões modernas do Python.

## Uso da Classe

A `DatabaseService` fornece métodos para inserir, recuperar, atualizar e excluir informações de cartões de crédito armazenadas no banco de dados SQLite.

## Métodos

### `__init__`

```python
def __init__(self)
```

Inicializa a instância do `DatabaseService` com o caminho para o banco de dados especificado em `Config.DATABASE_PATH` e chama o método `_create_table` para garantir que a tabela `credit_cards` exista.

---

### `_create_table`

```python
def _create_table(self)
```

Cria a tabela `credit_cards` no banco de dados, caso ela ainda não exista. A tabela armazena informações como o nome do cartão, número, data de expiração, banco emissor, validade e data de processamento.

---

### `_execute_query`

```python
def _execute_query(self, query: str, params: tuple = None) -> Optional[sqlite3.Cursor]
```

Executa uma query SQL com tratamento de erros.

#### Argumentos
- `query` (str): A consulta SQL a ser executada.
- `params` (tuple, opcional): Os parâmetros da consulta SQL.

#### Retorno
- `Optional[sqlite3.Cursor]`: Retorna o cursor da query se bem-sucedida; caso contrário, retorna `None`.

---

### `insert_card`

```python
def insert_card(self, card_info: Dict[str, str]) -> int
```

Insere um novo registro de cartão de crédito no banco de dados.

#### Argumentos
- `card_info` (Dict[str, str]): Um dicionário contendo as informações do cartão.

#### Retorno
- `int`: O ID do cartão inserido ou `None` se ocorrer um erro.

---

### `get_all_cards`

```python
def get_all_cards(self) -> List[Dict[str, str]]
```

Retorna todas as informações dos cartões armazenados no banco de dados.

#### Retorno
- `List[Dict[str, str]]`: Uma lista de dicionários com as informações de cada cartão.

---

### `get_card_by_id`

```python
def get_card_by_id(self, card_id: int) -> Optional[Dict[str, str]]
```

Retorna informações de um cartão específico baseado no ID.

#### Argumentos
- `card_id` (int): O ID do cartão desejado.

#### Retorno
- `Optional[Dict[str, str]]`: Um dicionário com as informações do cartão ou `None` se o cartão não for encontrado.

---

### `get_card_by_number`

```python
def get_card_by_number(self, card_number: int) -> Optional[Dict[str, str]]
```

Retorna informações de um cartão específico baseado no número do cartão.

#### Argumentos
- `card_number` (int): O número do cartão desejado.

#### Retorno
- `Optional[Dict[str, str]]`: Um dicionário com as informações do cartão ou `None` se o cartão não for encontrado.

---

### `execute_custom_query`

```python
def execute_custom_query(self, query: str) -> List[Dict[str, str]]
```

Executa uma consulta SQL personalizada e retorna os resultados.

#### Argumentos
- `query` (str): A consulta SQL a ser executada (apenas SELECT é permitido).

#### Retorno
- `List[Dict[str, str]]`: Uma lista de dicionários contendo os resultados da consulta.

#### Exceções
- `ValueError`: Lançada se a consulta SQL não começar com `SELECT`.

---

### `update_card`

```python
def update_card(self, card_id: int, card_info: Dict[str, str]) -> bool
```

Atualiza as informações de um cartão existente no banco de dados.

#### Argumentos
- `card_id` (int): O ID do cartão a ser atualizado.
- `card_info` (Dict[str, str]): Um dicionário com as informações atualizadas do cartão.

#### Retorno
- `bool`: `True` se a atualização for bem-sucedida; caso contrário, `False`.

---

### `delete_card`

```python
def delete_card(self, card_id: int) -> bool
```

Deleta um cartão do banco de dados com base no ID.

#### Argumentos
- `card_id` (int): O ID do cartão a ser deletado.

#### Retorno
- `bool`: `True` se a exclusão for bem-sucedida; caso contrário, `False`.

---

## Exemplo Completo de Uso

Abaixo está um exemplo completo de como usar o `DatabaseService` para gerenciar informações de cartões de crédito no banco de dados SQLite.

```python
from utils.Config import Config  # Certifique-se de que Config.DATABASE_PATH está corretamente configurado

# Inicialize o serviço de banco de dados
db_service = DatabaseService()

# Exemplo de dados do cartão
card_info = {
    "card_name": "GABRIEL LIMA",
    "card_number": "5032933437649846",
    "expiry_date": "09/25",
    "bank_name": "NY Bank",
    "is_valid": "True",
    "processed_at": "2024-11-12",
}

# Insere o cartão no banco de dados
card_id = db_service.insert_card(card_info)
print(f"Cartão inserido com ID: {card_id}")

# Recupera todos os cartões
cards = db_service.get_all_cards()
print("Cartões armazenados:", cards)

# Atualiza um cartão
update_info = {**card_info, "expiry_date": "10/25"}
is_updated = db_service.update_card(card_id, update_info)
print("Cartão atualizado:", is_updated)

# Deleta um cartão
is_deleted = db_service.delete_card(card_id)
print("Cartão deletado:", is_deleted)
```

### Observações
- **Configuração**: O caminho do banco de dados `Config.DATABASE_PATH` deve estar corretamente configurado para o `DatabaseService` funcionar.
- **Tratamento de Erros**: A classe lida com erros de execução SQL e retorna valores adequados em caso de falhas nas operações de banco de dados.
- **Consultas Personalizadas**: O método `execute_custom_query` aceita apenas consultas `SELECT` para garantir segurança e evitar operações de manipulação de dados indesejadas.


## Classe Config.py: 

In [65]:
"""
Módulo de configuração do aplicativo.

Este módulo carrega as variáveis de ambiente necessárias para a execução do aplicativo,
incluindo as credenciais para o Azure Document Intelligence, Azure Blob Storage e o caminho do banco de dados.
"""
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()


class Config:
    """Classe para armazenar as configurações do aplicativo."""

    # Configurações para o Azure Document Intelligence
    AZURE_DOC_INT_ENDPOINT: str = os.getenv("AZURE_DOC_INT_ENDPOINT")
    """Endpoint do Azure Document Intelligence."""
    AZURE_DOC_INT_KEY: str = os.getenv("AZURE_DOC_INT_KEY")
    """Chave do Azure Document Intelligence."""

    # Configurações para o Azure Blob Storage
    AZURE_STORAGE_CONNECTION: str = os.getenv("AZURE_STORAGE_CONNECTION")
    """String de conexão do Azure Blob Storage."""
    CONTAINER_NAME: str = os.getenv("CONTAINER_NAME")
    """Nome do container no Azure Blob Storage."""

    # Configurações para o banco de dados SQLite
    DATABASE_PATH: str = os.getenv("DATABASE_PATH", "data/cards.db")
    """Caminho do arquivo do banco de dados SQLite."""

    #Validação das configurações
    @classmethod
    def validate_config(cls):
        """Valida se todas as configurações necessárias foram fornecidas."""
        required_vars = ["AZURE_DOC_INT_ENDPOINT", "AZURE_DOC_INT_KEY", "AZURE_STORAGE_CONNECTION", "CONTAINER_NAME"]
        missing_vars = [var for var in required_vars if getattr(cls, var) is None]
        if missing_vars:
            raise ValueError(f"As seguintes variáveis de ambiente estão faltando: {', '.join(missing_vars)}")

#Valida a configuração ao criar a classe
Config.validate_config()

# Config

A classe `Config` carrega e gerencia as configurações essenciais para o funcionamento do aplicativo, incluindo credenciais para integração com o Azure Document Intelligence, Azure Blob Storage, e o caminho para o banco de dados SQLite.

## Sumário
- [Introdução](#introdução)
- [Carregamento de Variáveis de Ambiente](#carregamento-de-variáveis-de-ambiente)
- [Atributos da Classe Config](#atributos-da-classe-config)
  - [AZURE_DOC_INT_ENDPOINT](#azure_doc_int_endpoint)
  - [AZURE_DOC_INT_KEY](#azure_doc_int_key)
  - [AZURE_STORAGE_CONNECTION](#azure_storage_connection)
  - [CONTAINER_NAME](#container_name)
  - [DATABASE_PATH](#database_path)
- [Métodos](#métodos)
  - [`validate_config`](#validate_config)
- [Exemplo de Uso](#exemplo-de-uso)

## Introdução

A `Config` centraliza e simplifica a configuração do aplicativo, obtendo valores a partir de variáveis de ambiente carregadas com `dotenv`. Ela também valida a presença de variáveis essenciais para evitar erros em tempo de execução.

## Carregamento de Variáveis de Ambiente

O módulo utiliza o `dotenv` para carregar automaticamente as variáveis de ambiente do arquivo `.env`, que devem incluir valores como credenciais e endpoints necessários para o funcionamento do aplicativo.

## Atributos da Classe Config

### `AZURE_DOC_INT_ENDPOINT`

```python
AZURE_DOC_INT_ENDPOINT: str = os.getenv("AZURE_DOC_INT_ENDPOINT")
```

O endpoint do serviço Azure Document Intelligence, necessário para interações com a API.

---

### `AZURE_DOC_INT_KEY`

```python
AZURE_DOC_INT_KEY: str = os.getenv("AZURE_DOC_INT_KEY")
```

A chave de acesso ao Azure Document Intelligence. É utilizada para autenticação ao interagir com os serviços de análise de documentos.

---

### `AZURE_STORAGE_CONNECTION`

```python
AZURE_STORAGE_CONNECTION: str = os.getenv("AZURE_STORAGE_CONNECTION")
```

A string de conexão do Azure Blob Storage. É usada para se conectar ao serviço de armazenamento de blobs do Azure.

---

### `CONTAINER_NAME`

```python
CONTAINER_NAME: str = os.getenv("CONTAINER_NAME")
```

O nome do container onde os arquivos serão armazenados no Azure Blob Storage.

---

### `DATABASE_PATH`

```python
DATABASE_PATH: str = os.getenv("DATABASE_PATH", "data/cards.db")
```

O caminho do arquivo de banco de dados SQLite. Caso a variável de ambiente não esteja definida, o valor padrão será `"data/cards.db"`.

---

## Métodos

### `validate_config`

```python
@classmethod
def validate_config(cls)
```

Valida se todas as variáveis de ambiente essenciais foram fornecidas. Se alguma variável obrigatória estiver ausente, lança uma exceção `ValueError` com uma mensagem informando quais variáveis estão faltando.

#### Exceções
- `ValueError`: Lançada se uma ou mais variáveis essenciais não forem encontradas.

---

## Exemplo de Uso

### Configuração no Arquivo `.env`

Para que a classe `Config` funcione corretamente, o arquivo `.env` deve conter as seguintes variáveis:

```dotenv
AZURE_DOC_INT_ENDPOINT=https://seu-endpoint-azure-doc-intelligence
AZURE_DOC_INT_KEY=sua-chave-azure-doc-intelligence
AZURE_STORAGE_CONNECTION=sua-string-de-conexao-azure-blob-storage
CONTAINER_NAME=nome-do-container
DATABASE_PATH=caminho/para/seu/cards.db
```

### Validação e Acesso às Configurações

Após definir as variáveis no `.env`, ao instanciar ou acessar `Config`, o método `validate_config` garante que todas as configurações críticas estejam presentes.

```python
from utils.Config import Config

# Acessando uma configuração específica
endpoint = Config.AZURE_DOC_INT_ENDPOINT
print(f"Endpoint do Azure Document Intelligence: {endpoint}")

# Todas as configurações essenciais estão validadas ao inicializar
try:
    Config.validate_config()
    print("Configurações validadas com sucesso.")
except ValueError as e:
    print(f"Erro de configuração: {e}")
```

### Observações
- **Variáveis Essenciais**: As variáveis de ambiente `AZURE_DOC_INT_ENDPOINT`, `AZURE_DOC_INT_KEY`, `AZURE_STORAGE_CONNECTION`, e `CONTAINER_NAME` são obrigatórias para o funcionamento do aplicativo.
- **Variável Opcional**: `DATABASE_PATH` tem um valor padrão, então sua ausência não impede a execução do aplicativo.
- **Validação de Configuração**: `validate_config` é chamado na inicialização da classe para verificar a presença de todas as variáveis essenciais, evitando erros inesperados em produção.


## Frontend: 

In [ ]:
import datetime
import streamlit as st
import pandas as pd
import requests

from streamlit_lottie import st_lottie
from services.credit_card_service import CreditCardValidator
from services.blob_service import BlobStorageService
from services.data_base import DatabaseService
from utils.Config import Config

# Inicialização de serviços globais
CREDIT_CARD_VALIDATOR = CreditCardValidator()
BLOB_STORAGE_SERVICE = BlobStorageService()
DATABASE_SERVICE = DatabaseService()

def carregar_lottie(url: str):
    """Carrega animação do Lottie Files."""
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

def render_sidebar():
    """
    Renderiza a barra lateral do aplicativo com navegação e informações do desenvolvedor.
    
    Returns:
        str: Página selecionada no menu
    """
    menu_options = {
        "Início": "🏠",
        "Análise de Cartão": "💳",
        "Consulta Banco de Dados": "🔍",
        "Documentação": "📚",
        "Sobre": "ℹ️"
    }
    
    selected_page = st.sidebar.radio(
        "Menu", 
        list(menu_options.keys()), 
        format_func=lambda x: f"{x} {menu_options[x]}"
    )
    
    st.sidebar.markdown("---")
    st.sidebar.markdown("### 👨‍💻 Desenvolvido por:\n**Julio Okuda**")
    
    st.sidebar.markdown("""
        <div class="social-links">
            <a href="https://github.com/Jcnok" target="_blank">
                <img src="https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white" />
            </a>
            <a href="https://linkedin.com/in/juliookuda" target="_blank">
                <img src="https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white" />
            </a>
        </div>
    """, unsafe_allow_html=True)
    
    return selected_page

def process_card_analysis(uploaded_file):
    """
    Processa a imagem de cartão de crédito carregada.
    
    Args:
        uploaded_file (UploadedFile): Arquivo de imagem carregado
    
    Returns:
        tuple: Informações do cartão e resultado da validação, ou None
    """
    try:
        st.image(uploaded_file, caption="Imagem do Cartão", use_column_width=True)
        
        with st.spinner("Processando..."):
            file_name = uploaded_file.name
            file = uploaded_file.getvalue()
            blob_url = BLOB_STORAGE_SERVICE.upload_blob(file, file_name)
            
            if not blob_url:
                st.error("Erro ao carregar imagem para o Blob Storage.")
                return None
            
            card_info = CREDIT_CARD_VALIDATOR.detect_credit_card_info_from_url(blob_url)
            if not card_info:
                st.error("Não foi possível analisar o cartão.")
                return None
            
            validation_result = CREDIT_CARD_VALIDATOR.validate_card_info(card_info)
            return card_info, validation_result
    
    except Exception as e:
        st.error(f"Erro durante análise do cartão: {e}")
        return None

def database_query_page():
    """
    Página de consulta ao banco de dados que exibe todos os dados automaticamente
    e permite executar queries personalizadas com exemplos prontos.
    """
    st.title("🔍 Consulta Banco de Dados")

    # Exibe automaticamente todos os dados ao carregar a página
    st.subheader("📊 Todos os Registros na Tabela `credit_cards`")
    try:
        all_data = DATABASE_SERVICE.get_all_cards()
        if all_data:
            df_all = pd.DataFrame(all_data)
            st.dataframe(df_all, use_container_width=True)

            # Botão para exportar todos os dados como CSV
            csv_all = df_all.to_csv(index=False)
            st.download_button(
                label="💾 Baixar Todos os Dados (CSV)",
                data=csv_all,
                file_name="todos_os_dados_credit_cards.csv",
                mime="text/csv",
            )
        else:
            st.info("⚠️ Nenhum dado encontrado na tabela `credit_cards`.")
    except Exception as e:
        st.error(f"Erro ao carregar todos os dados: {e}")

    # Permitir consultas personalizadas
    st.markdown("---")
    st.subheader("🔎 Consultas Personalizadas")
    st.markdown(
        """
        **Exemplos de Queries:**
        - Todos os registros: `SELECT * FROM credit_cards`
        - Cartões por id: `SELECT * FROM credit_cards WHERE id = 1`
        - Cartões por nome: `SELECT * FROM credit_cards WHERE card_name = "GABRIEL LIMA"`
        - Cartões emitidos pelo banco: `SELECT * FROM credit_cards WHERE bank_name = 'Banco X'`
        """
    )

    # Caixa de texto para consulta SQL
    query = st.text_area(
        "Digite sua consulta SQL:",
        placeholder="Exemplo: SELECT * FROM credit_cards WHERE bank_name = 'Banco X'",
        height=150,
    )

    if st.button("Executar Consulta"):
        try:
            # Executa a consulta personalizada
            results = DATABASE_SERVICE.execute_custom_query(query)

            if results:
                # Converte resultados para DataFrame
                df_results = pd.DataFrame(results)
                st.dataframe(df_results, use_container_width=True)

                # Botão para exportar os resultados como CSV
                csv_results = df_results.to_csv(index=False)
                st.download_button(
                    label="💾 Baixar Resultados (CSV)",
                    data=csv_results,
                    file_name="resultados_consulta.csv",
                    mime="text/csv",
                )
            else:
                st.info("🔍 Nenhum resultado encontrado para a consulta.")
        except ValueError as ve:
            st.error(f"❌ Erro de validação: {ve}")
        except Exception as e:
            st.error(f"❌ Erro ao executar a consulta: {e}")


def home_page():
    """
    Renderiza a página inicial com descrição do projeto.
    """
    st.title("🌟 Simplificando a Validação de Cartões no E-commerce")
    # Carregar animação
    lottie_translate = carregar_lottie(
        "https://lottie.host/c6d163ab-0ab8-49aa-8c50-332eb30e3774/kktucbjshh.json"
    )
    st_lottie(lottie_translate, height=300)
    st.markdown("""
      Já parou para pensar como algumas plataformas de e-commerce utilizam tecnologias 
      avançadas para facilitar compras e prevenir fraudes? Lembra daquele momento mágico 
      em que, ao finalizar uma compra, em vez de digitar todos os dados do cartão, você 
      pode simplesmente enviar uma foto?
      
      #### 💡 Nossa Proposta
      Este projeto demonstra exatamente como essa mágica acontece! Utilizando a 
      Inteligência Artificial da Azure, implementamos um sistema de validação 
      de cartões que torna esse processo não só possível, mas também extremamente 
      simples.
      
      #### 🚀 Como Funciona
      1. Faça upload da imagem do cartão
      2. A IA analisa os dados instantaneamente
      3. Receba a validação em segundos
      4. Dados são armazenados para análises futuras
      
      #### 🎯 Benefícios
      - Detecção automática das informações sem digitação
      - Interface intuitiva e amigável
      - Armazenamento para análises futuras
      - Consultas facilitadas com exportação para CSV
      
      #### 🔍 Explorando o Projeto
      Este é um projeto demonstrativo que utiliza tecnologias de ponta da Azure 
      para mostrar como implementar validação de cartões de forma eficiente. 
      Embora seja uma POC (Prova de Conceito), já inclui os principais elementos 
      necessários para um sistema completo:
      
      - Extração precisa de dados do cartão
      - Validação em tempo real
      - Armazenamento estruturado
      - Interface para análise de dados
      
      #### 🎯 Objetivo
      Demonstrar na prática como as tecnologias modernas podem ser aplicadas 
      para criar soluções que melhoram significativamente a experiência do 
      usuário em transações financeiras.
    """)

def documentation_page():
    """
    Renderiza a página de documentação.
    """
    st.title("📚 Documentação")
    st.markdown("""
      # Documentação do Credit Card Analyzer

      ## Visão Geral

      Este aplicativo utiliza a API do Azure Document Intelligence para extrair informações de cartões de crédito a partir de imagens.  Os dados extraídos são então validados e persistidos em um banco de dados SQLite.

      ##  Funcionalidades Principais

      * **Upload de Imagem:** Permite ao usuário carregar uma imagem de um cartão de crédito.
      * **Análise de Imagem:** Usa a API do Azure Document Intelligence para detectar e extrair informações como número do cartão, data de validade, nome do titular e nome do banco.
      * **Validação de Cartão:** Realiza uma validação básica do número e data de validade do cartão.
      * **Armazenamento de Dados:** Armazena as informações do cartão (incluindo o resultado da validação) em um banco de dados SQLite.
      * **Consulta de Dados:** Permite consultar os dados armazenados no banco de dados utilizando consultas SQL.
      * **Exportação de Dados:** Permite exportar os resultados das consultas para um arquivo CSV.

      ##  Arquitetura

      O aplicativo segue uma arquitetura de três camadas:

      1. **Frontend (Streamlit):** Interface do usuário para interação com o usuário.
      2. **Backend (Python):** Lógica de negócio, incluindo a interação com os serviços da Azure e o banco de dados.
      3. **Azure Services:** Azure Document Intelligence e Azure Blob Storage.

      ##  Tecnologias Utilizadas

      * **Streamlit:** Framework Python para criar aplicações web.
      * **Python:** Linguagem de programação.
      * **Azure Document Intelligence:** Serviço da Azure para extração de informações de documentos.
      * **Azure Blob Storage:** Serviço da Azure para armazenamento de objetos de dados binários.
      * **SQLite:** Sistema de gerenciamento de banco de dados relacional.
    """)

def about_page():
    """
    Renderiza a página sobre o projeto.
    """
    st.title("ℹ️ Sobre")
    st.markdown("""
      ### 🎯 Projeto Credit Card Analyzer

      Este projeto é uma Prova de Conceito (POC) desenvolvida como parte do 
      [Bootcamp Microsoft Certification Challenge #1 - AI 102](https://www.dio.me/bootcamp/microsoft-ai-102). O objetivo é demonstrar a aplicação 
      prática de conceitos modernos de desenvolvimento e integração com 
      serviços em nuvem da Azure.

      #### 🛠️ Tecnologias Utilizadas
      - **Frontend**: Streamlit
      - **Backend**: Python
      - **Cloud**: Azure Services
      - **Database**: SQLite
      - **Version Control**: Git

      #### 🌟 Características
      - Interface intuitiva
      - Processamento de imagem e validação
      - Armazenamento em banco de dados
      - Análise e exportação de dados
      - Integração com serviços Azure (Document Intelligence e Blob Storage)

      #### 👨‍💻 Desenvolvimento
      Desenvolvido por **Julio Okuda** como parte do projeto final do bootcamp,
      demonstrando a aplicação prática dos conceitos aprendidos durante o bootcamp.

      #### 📝 Nota
      Este é um projeto educacional e demonstrativo, não devendo ser utilizado
      em ambiente de produção sem as devidas adaptações e medidas de segurança.
    """)

def card_analysis_page():
    """
    Página para análise de cartões de crédito.
    """
    st.title("💳 Análise de Cartão")
    uploaded_file = st.file_uploader("Carregue a imagem do cartão", type=["jpg", "jpeg", "png"])
    
    if uploaded_file and st.button("💳 Analisar Cartão"):
        result = process_card_analysis(uploaded_file)
        if result:
            card_info, validation_result = result
            
            st.write("Informações do Cartão:")
            st.write(card_info)
            
            if validation_result["is_valid"]:
                st.success("✅ Cartão Válido")
                existing_card = DATABASE_SERVICE.get_card_by_number(card_info["card_number"])
                
                if existing_card:
                    st.info(f"Cartão já existe no banco de dados. ID: {existing_card['id']}")
                else:
                    card_info["is_valid"] = validation_result["is_valid"]
                    card_info["processed_at"] = datetime.datetime.now().isoformat()
                    DATABASE_SERVICE.insert_card(card_info)
                    st.success("Cartão inserido no banco de dados!")
            else:
                st.error("❌ Cartão Inválido")

def main():
    """
    Ponto de entrada para o aplicativo Credit Card Analyzer.
    """
    st.set_page_config(
        page_title="Credit Card Analyzer", 
        page_icon="💳", 
        layout="wide"
    )
    
    # Mapeamento de páginas
    page_handlers = {
        "Início": home_page,
        "Análise de Cartão": card_analysis_page,
        "Consulta Banco de Dados": database_query_page,
        "Documentação": documentation_page,
        "Sobre": about_page
    }
    
    # Renderiza a página selecionada
    selected_page = render_sidebar()
    handler = page_handlers.get(selected_page)
    
    if handler:
        handler()

if __name__ == "__main__":
    main()